In [2]:
pip install nltk

  Obtaining dependency information for nltk from https://files.pythonhosted.org/packages/a6/0a/0d20d2c0f16be91b9fa32a77b76c60f9baf6eba419e5ef5deca17af9c582/nltk-3.8.1-py3-none-any.whl.metadata
  Obtaining dependency information for click from https://files.pythonhosted.org/packages/00/2e/d53fa4befbf2cfa713304affc7ca780ce4fc1fd8710527771b58311a3229/click-8.1.7-py3-none-any.whl.metadata
  Obtaining dependency information for regex>=2021.8.3 from https://files.pythonhosted.org/packages/a8/01/18232f93672c1d530834e2e0568a80eaab1df12d67ae499b1762ab462b5c/regex-2023.12.25-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ----------------------------- ---------- 30.7/42.0 kB ? eta -:--:--
     ----------------------------- ---------- 30.7/42.0 kB ? eta -:--:--
     ----------------------------- ---------- 30.7/42.0 kB ? eta -:--:--
     ----------------------------- ---------- 30.7/42.0 kB ? eta -:--:--
     ------------------------


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import nltk
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
lemmatizer = nltk.stem.WordNetLemmatizer()

# Download required NLTK data
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to C:\Users\MR
[nltk_data]     LAWAL\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to C:\Users\MR
[nltk_data]     LAWAL\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to C:\Users\MR
[nltk_data]     LAWAL\AppData\Roaming\nltk_data...


True

In [4]:
data = pd.read_csv('Samsung Dialog.txt', sep = ':', header = None)
data.head()

,0,1
0,Customer,"Hi, I'm looking to buy a new phone, and I'm i..."
1,Sales Agent,"Great, we have a wide range of Samsung phones..."
2,Customer,"Well, I want a phone with a good camera, long..."
3,Sales Agent,Absolutely. We have a lot of great options th...
4,Customer,"No, I haven't. Tell me more about it."


In [5]:
cust = data.loc[data[0] == 'Customer']
sales = data.loc[data[0] == 'Sales Agent']

df = pd.DataFrame()
df['Question'] = cust[1].reset_index(drop = True)
df['Answer'] = sales[1].reset_index(drop = True)

df.head()

,Question,Answer
0,"Hi, I'm looking to buy a new phone, and I'm i...","Great, we have a wide range of Samsung phones..."
1,"Well, I want a phone with a good camera, long...",Absolutely. We have a lot of great options th...
2,"No, I haven't. Tell me more about it.",The Galaxy S21 Ultra has a 108-megapixel came...
3,That sounds great. How much does it cost?,"The Galaxy S21 Ultra starts at $1,199, but we..."
4,"Okay, I'm interested. But I have a few more q...",The Galaxy S21 Ultra comes with a standard on...


In [6]:
def preprocess_text(text):
    # Identifies all sentences in the data
    sentences = nltk.sent_tokenize(text)

    # Tokenize and lemmatize each word in each sentence
    preprocessed_sentences = []
    for sentence in sentences:
        tokens = [lemmatizer.lemmatize(word.lower()) for word in nltk.word_tokenize(sentence) if word.isalnum()]
        # Turns to basic root - each word in the tokenized word found in the tokenized sentence - if they are all alphanumeric
        # The code above does the following:
        # Identifies every word in the sentence
        # Turns it to a lower case
        # Lemmatizes it if the word is alphanumeric

        preprocessed_sentence = ' '.join(tokens)
        preprocessed_sentences.append(preprocessed_sentence)

    return ' '.join(preprocessed_sentences)


df['tokenized Questions'] = df['Question'].apply(preprocess_text)
df.head()# Define a function for text preprocessing (including lemmatization)


,Question,Answer,tokenized Questions
0,"Hi, I'm looking to buy a new phone, and I'm i...","Great, we have a wide range of Samsung phones...",hi i looking to buy a new phone and i interest...
1,"Well, I want a phone with a good camera, long...",Absolutely. We have a lot of great options th...,well i want a phone with a good camera long ba...
2,"No, I haven't. Tell me more about it.",The Galaxy S21 Ultra has a 108-megapixel came...,no i have tell me more about it
3,That sounds great. How much does it cost?,"The Galaxy S21 Ultra starts at $1,199, but we...",that sound great how much doe it cost
4,"Okay, I'm interested. But I have a few more q...",The Galaxy S21 Ultra comes with a standard on...,okay i interested but i have a few more questi...


In [7]:
x = df['tokenized Questions'].to_list()
x

['hi i looking to buy a new phone and i interested in samsung phone',
 'well i want a phone with a good camera long battery life and plenty of storage',
 'no i have tell me more about it',
 'that sound great how much doe it cost',
 'okay i interested but i have a few more question what kind of warranty come with the phone',
 'that good to know and what about the operating system doe it come with the latest version of android',
 'okay that good but i also interested in some of the other samsung phone can you tell me more about the galaxy a52',
 'that sound like a good option for me how much doe it cost',
 'okay i think about it but can you also tell me about the galaxy z fold2 i heard a lot about it and i curious',
 'that sound really cool but it also sound expensive how much doe it cost',
 'hmm i not sure that a lot of money for a phone',
 'okay can you tell me more about the galaxy a72',
 'that sound like a great option for me how much doe it cost',
 'okay i definitely consider it but

In [8]:
# Vectorize corpus
tfidf_vectorizer = TfidfVectorizer()
corpus = tfidf_vectorizer.fit_transform(x)
# TDIDF is a numerical statistic used to evaluate how important a word is to a document in a collection or corpus.
# The TfidfVectorizer calculates the Tfidf values for each word in the corpus and uses them
# to create a matrix where each row represents a document and each column represents a word.
# The cell values in the matrix correspond to the importance of each word in each document.

In [9]:
print(corpus[0])

  (0, 71)	0.3011232459093611
  (0, 41)	0.2700934382194581
  (0, 42)	0.2460248494975018
  (0, 5)	0.22635938061087743
  (0, 67)	0.4194649271573511
  (0, 58)	0.34485730351794186
  (0, 11)	0.34485730351794186
  (0, 84)	0.2700934382194581
  (0, 51)	0.34485730351794186
  (0, 37)	0.34485730351794186


In [10]:
user = input('pls ask your question: ')
user

'can you tell me about samsung galazy fold z'

In [11]:
pre_user = preprocess_text(user)
pre_user

'can you tell me about samsung galazy fold z'

In [12]:
vect_user =tfidf_vectorizer.transform([pre_user])
print(vect_user)

  (0, 91)	0.3940483529904176
  (0, 79)	0.3940483529904176
  (0, 71)	0.5241979315259283
  (0, 54)	0.3400313973828437
  (0, 14)	0.4282821700500242
  (0, 2)	0.3400313973828437


In [13]:
similarity_scores = cosine_similarity(vect_user,corpus)
similarity_scores

array([[0.15784818, 0.        , 0.35106848, 0.        , 0.        ,
        0.05585161, 0.48547037, 0.08326295, 0.44750028, 0.        ,
        0.        , 0.58149748, 0.08187777, 0.37576539, 0.07786208,
        0.06522357, 0.        ]])

In [14]:
most_similarity_index = similarity_scores.argmax()
most_similarity_index

11

In [15]:
df['Answer'].iloc[most_similarity_index]

' Sure, the Galaxy A72 is another great mid-range phone that has a lot of the same features as the A52, but with a bigger display, bigger battery, and more cameras. It has a 6.7-inch Super AMOLED display, a 5,000mAh battery, and four cameras, including a 64-megapixel main camera, a 12-megapixel ultra-wide camera, an 8-megapixel telephoto camera, and a 5-megapixel macro camera. It also has up to 256GB of storage and runs on Android 11.'

In [16]:
def collector():
    user = input('Please ask your question')
    pre_user = preprocess_text(user)
    vect_user = tfidf_vectorizer.transform([pre_user])
    similarity_scores = cosine_similarity(vect_user, corpus)
    most_similarity_index = similarity_scores.argmax()
    return(df['Answer'].iloc[most_similarity_index])

collector()

' Sure, the Galaxy A72 is another great mid-range phone that has a lot of the same features as the A52, but with a bigger display, bigger battery, and more cameras. It has a 6.7-inch Super AMOLED display, a 5,000mAh battery, and four cameras, including a 64-megapixel main camera, a 12-megapixel ultra-wide camera, an 8-megapixel telephoto camera, and a 5-megapixel macro camera. It also has up to 256GB of storage and runs on Android 11.'

In [17]:
def get_response(user_input):
    user_input_processed = preprocess_text(user_input) # ....................... Preprocess the user's input using the preprocess_text function

    user_input_vector = tfidf_vectorizer.transform([user_input_processed])# .... Vectorize the preprocessed user input using the TF-IDF vectorizer

    similarity_scores = cosine_similarity(user_input_vector, corpus) # .. Calculate the score of similarity between the user input vector and the corpus (df) vector

    most_similar_index = similarity_scores.argmax() # ..... Find the index of the most similar question in the corpus (df) based on cosine similarity

    return df['Answer'].iloc[most_similar_index] # ... Retrieve the corresponding answer from the df DataFrame and return it as the chatbot's response

# create greeting list
greetings = ["Hey There.... I am a creation of Ehiz Danny Agba Coder.... How can I help",
            "Hi Human.... How can I help",
            'Twale baba nla, wetin dey happen nah',
            'How far Alaye, wetin happen'
            "Good Day .... How can I help",
            "Hello There... How can I be useful to you today",
            "Hi GomyCode Student.... How can I be of use"]

exits = ['thanks bye', 'bye', 'quit', 'exit', 'bye bye', 'close']
farewell = ['Thanks....see you soon', 'Babye, See you soon', 'Bye... See you later', 'Bye... come back soon']

random_farewell = random.choice(farewell) # ---------------- Randomly select a farewell message from the list
random_greetings = random.choice(greetings) # -------- Randomly select greeting message from the list

# Test your chatbot
while True:
    user_input = input("You: ")
    if user_input.lower() in exits:
        print(f"\nChatbot: {random_farewell}!")
        break
    if user_input.lower() in ['hi', 'hello', 'hey', 'hi there']:
        print(f"\nChatbot: {random_greetings}!")
    else:
        response = get_response(user_input)
        print(f"\nChatbot: {response}")


Chatbot:  The Galaxy S21 Ultra starts at $1,199, but we have some great financing options that make it affordable for everyone.

Chatbot:  Sure, the Galaxy A72 is another great mid-range phone that has a lot of the same features as the A52, but with a bigger display, bigger battery, and more cameras. It has a 6.7-inch Super AMOLED display, a 5,000mAh battery, and four cameras, including a 64-megapixel main camera, a 12-megapixel ultra-wide camera, an 8-megapixel telephoto camera, and a 5-megapixel macro camera. It also has up to 256GB of storage and runs on Android 11.

Chatbot:  Great, we have a wide range of Samsung phones to choose from. What features are you looking for in a phone?

Chatbot: Hi GomyCode Student.... How can I be of use!

Chatbot: Babye, See you soon!
